In [1]:
import os
import sys

ROOT_DIR = os.path.abspath("__file__" + "/../../")
sys.path.insert(0, f"{ROOT_DIR}")

# Single Vertex Spinfoam

In [2]:
import numpy as np

gridLength = 8
spin_j = (gridLength - 1) / 2
env_name = f"single vertex spinfoam/j={float(spin_j)}"
evaluate_after_n_states = int(1e5)

def get_rewards_for_single_vertex_spinfoam(gridLength, root=ROOT_DIR):
    spin_j = (gridLength - 1) / 2

    vertex = np.load(f"{root}/data/EPRL_vertices/Python/Dl_20/vertex_j_{float(spin_j)}.npz")
    sq_ampl = vertex**2
    return sq_ampl / np.sum(sq_ampl)

rewards = get_rewards_for_single_vertex_spinfoam(gridLength)

In [3]:
from src.grid_environments.base import BaseGrid
from src.trainers.trainer import base_train_gfn

generated_data_dir = f"{ROOT_DIR}/data/GFN/{env_name}"

losses = base_train_gfn(
    BaseGrid(grid_rewards=rewards),
    generated_data_dir,
    hidden_dim=256,
    n_hidden_layers=2,
    activation_fn="relu",
    exploration_rate=0.5,
    learning_rate=0.0005,
    batch_size=int(1e3),
    n_iterations=int(1e3),
    evaluation_batch_size=int(1e5),
    generate_samples_every_m_training_samples=evaluate_after_n_states,
)

  0%|                                                                                                                                              | 0/1000 [00:00<?, ?it/s]/home/josephb/anaconda3/envs/gfn/lib/python3.11/site-packages/torch/autograd/__init__.py:200: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [04:49<00:00,  3.46it/s, loss=1.36]


In [4]:
from src.MCMC.MCMC import VertexMCMC

iterationsNumber = int(1e6)
batchSize = evaluate_after_n_states
burnFactor = 0
deviation = 0.8
dimensions = 5

verbosity = 1

drawsFolder = f"{ROOT_DIR}/data/MCMC/{env_name}"
os.makedirs(drawsFolder, exist_ok=True)

VertexMCMC(
    gridLength = gridLength, iterationsNumber = iterationsNumber, batchSize = batchSize, 
    deviation = deviation, burnFactor = burnFactor, verbosity = verbosity, 
    drawsFolder = drawsFolder, mean = 0., rewardFunction=get_rewards_for_single_vertex_spinfoam, 
    dimensions=dimensions 
)

# Grid Peaks from GFN Paper

In [5]:
from src.MCMC.MCMC import grid_rewards_2d

gridLength = 13
dimensions = 2
env_name = f"GFN Paper Grid Peaks/grid_length={gridLength}, grid_dim={dimensions}"
evaluate_after_n_states = int(1e5)

gfn_paper_rewards = grid_rewards_2d(gridLength)

In [ ]:
generated_data_dir = f"{ROOT_DIR}/data/GFN/{env_name}"

losses = base_train_gfn(
    BaseGrid(grid_rewards=gfn_paper_rewards),
    generated_data_dir,
    hidden_dim=256,
    n_hidden_layers=2,
    activation_fn="relu",
    exploration_rate=0.5,
    learning_rate=0.0005,
    batch_size=int(1e3),
    n_iterations=int(1e3),
    evaluation_batch_size=int(1e5),
    generate_samples_every_m_training_samples=evaluate_after_n_states,
)

 41%|████████████████████████████████████████████████▋                                                                     | 413/1000 [01:22<01:24,  6.95it/s, loss=0.00172]

In [ ]:
iterationsNumber = int(1e6)
batchSize = evaluate_after_n_states
burnFactor = 0
deviation = 0.8

verbosity = 1

drawsFolder = f"{ROOT_DIR}/data/MCMC/{env_name}"
os.makedirs(drawsFolder, exist_ok=True)

VertexMCMC(
    gridLength = gridLength, iterationsNumber = iterationsNumber, batchSize = batchSize, 
    deviation = deviation, burnFactor = burnFactor, verbosity = verbosity, 
    drawsFolder = drawsFolder, mean = 0., rewardFunction=grid_rewards_2d, 
    dimensions=dimensions 
)